In [1]:
!pip install -U /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -rf /kaggle/input/sentence-transformers-222/sentence-transformers /kaggle/working/sentence-transformers
!pip install -U /kaggle/working/sentence-transformers
!pip install -U /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl

!pip install --no-index --no-deps /kaggle/input/llm-accelerate-whl/accelerate-0.22.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-accelerate-whl/bitsandbytes-0.41.1-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-accelerate-whl/datasets-2.14.4-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-accelerate-whl/peft-0.5.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-accelerate-whl/transformers-4.32.0-py3-none-any.whl
!pip install --no-index --no-deps /kaggle/input/llm-accelerate-whl/trl-0.6.0-py3-none-any.whl

Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing ./sentence-transformers
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126134 sha256=72a97bf3ec9ab6ebf77b3fe7181e23b73175ee4596e4702e142f396e1968d3eb
  Stored in directory: /root/.cache/pip/wheels/6c/ea/76/d9a930b223b1d3d5d6aff69458725316b0fe205b854faf1812
Successfully built sentence-transformers
Processing /kaggle/input/blingfire-018/blingfire-0.1.8-py3-none-any.whl
Processing /kaggle/input/llm-accelerate-whl/accelerate-0.22.0-py3-none-any.whl
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3
Processing /kaggle/input/llm-accelerate-whl/bitsandbytes-0.41.1-py3-none-any.whl
Processing /kaggle/input/llm-accelerate-whl/datasets-2.14.4-

In [2]:
# Future imports
from __future__ import annotations

# Python standard library imports
from collections.abc import Iterable
import ctypes
import os
import gc
import re
from typing import Optional, Union
import warnings

# Third-party imports
import blingfire as bf
import faiss
from faiss import read_index, write_index
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
import torch
from torch.utils.data import DataLoader
from scipy.special import softmax
import torch.nn as nn

# Datasets library
from datasets import load_dataset, Dataset

# Pandas and Numpy
import pandas as pd
import numpy as np

# tqdm module for progress bars
from tqdm.auto import tqdm

# Import transformers components
from transformers import (
    AutoModel, 
    AutoModelForSequenceClassification, 
    AutoModelForMultipleChoice,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup, 
    get_linear_schedule_with_warmup, 
    Adafactor, 
    AdamW
)
from transformers.activations import ACT2FN
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

# Import sentence transformers
from sentence_transformers import SentenceTransformer

# Peft imports
from peft import (
    LoraConfig, 
    prepare_model_for_kbit_training, 
    get_peft_model, 
    PeftModel, 
    PeftConfig, 
    TaskType,
    PeftModelForSequenceClassification
)

# Load library
libc = ctypes.CDLL("libc.so.6")

# Set warnings to ignore
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
llama_model = AutoModelForSequenceClassification.from_pretrained(
    "/kaggle/input/llama2-7b-hf/Llama2-7b-hf",
    load_in_4bit=True,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
llama_model.config.use_cache = False

llama_model.config.pad_token_id = llama_model.config.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/llama2-7b-hf/Llama2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df_train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
df_test  = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
df_samp = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/sample_submission.csv')
df_extra = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv')
df_extra_RO = pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv')

In [5]:
# We have two dataframes named df_train and df_extra with the same number of rows
# Concatenate them
concatenated_df = pd.concat([df_train])
df_train.reset_index(inplace=True, drop=True)
df_train.shape

(200, 8)

In [6]:
import random
import pandas as pd
from transformers import pipeline, AutoTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/bert-base-uncased")

# Initialize the pipeline for masked language modeling using BERT
mlm_fill_mask = pipeline(task="fill-mask", model="/kaggle/input/bert-base-uncased")

# Function to perform data augmentation using different techniques
def augment_data(original_df, num_augmented_rows):
    augmented_data = []
    original_rows = original_df.shape[0]

    # Function for contextual word embeddings augmentation
    def contextual_embeddings(text):
        # Tokenize the text
        tokenized_text = tokenizer(text, return_tensors="pt")

        # Find masked positions in the tokenized text
        masked_positions = [i for i, token in enumerate(tokenized_text["input_ids"][0]) if token == tokenizer.mask_token_id]

        # If no masked positions found, return the original text
        if not masked_positions:
            return text

        # Randomly select one of the masked positions
        random_masked_position = random.choice(masked_positions)

        # Predict the masked word using masked language modeling
        masked_text = text.replace("[MASK]", tokenizer.mask_token)
        predicted_word = mlm_fill_mask(masked_text)[0]["token_str"]

        # Replace the masked word in the text with the predicted word
        augmented_text = text.replace(tokenizer.mask_token, predicted_word, 1)

        return augmented_text

    # Function for synonym replacement augmentation
    def augment_with_synonyms(text):
        # You can use your own synonym replacement logic here
        # For simplicity, let's assume we have a predefined dictionary of synonyms
        synonym_dict = {
            "good": ["excellent", "great", "superb", "fine"],
            "bad": ["poor", "terrible", "awful", "horrible"]
            # Add more synonyms as needed
        }

        words = text.split()
        augmented_words = [synonym_dict[word][random.randint(0, len(synonym_dict[word]) - 1)] if word in synonym_dict else word for word in words]
        return " ".join(augmented_words)

    for _ in range(num_augmented_rows):
        original_row = original_df.iloc[random.randint(0, original_rows - 1)]
        augmented_row = original_row.copy()

        # Apply augmentation techniques to "prompt"
        augmented_row["prompt"] = contextual_embeddings(original_row["prompt"])

        # Apply synonym replacement to answer choices (A, B, C, D, E)
        for choice in ["A", "B", "C", "D", "E"]:
            augmented_row[choice] = augment_with_synonyms(original_row[choice])

        augmented_data.append(augmented_row)

    return augmented_data

# Assuming you have a concatenated_df dataframe with columns: "prompt", "A", "B", "C", "D", "E", and "answer"
# Set the desired number of rows
desired_rows = 500

# Calculate the number of rows needed to achieve the desired total rows
additional_rows = desired_rows - concatenated_df.shape[0]

# Augment the data
augmented_data = augment_data(concatenated_df, additional_rows)

# Convert the augmented data to a dataframe
augmented_df = pd.DataFrame(augmented_data)

# Concatenate the original dataframe with the augmented dataframe to get the final dataframe with 3000 rows
final_df = pd.concat([concatenated_df, augmented_df], ignore_index=True)

In [7]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 9,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df

In [8]:
def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df

In [9]:
def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 9,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

In [10]:
SIM_MODEL = '/kaggle/input/sentencetransformers-allminilml6v2/sentence-transformers_all-MiniLM-L6-v2'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 32
WIKI_PATH = "/kaggle/input/wikipedia-20230701"
wiki_files = os.listdir(WIKI_PATH)

In [11]:
trn = pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv").drop("id", 1)
trn.head()


,prompt,A,B,C,D,E
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


In [12]:
## Combine all answers
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)


## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

In [13]:
model = SentenceTransformer(SIM_MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half()

In [14]:
sentence_index = read_index("/kaggle/input/wikipedia-2023-07-faiss-index/wikipedia_202307.index")


In [15]:
prompt_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()
_ = gc.collect()

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [16]:
search_score, search_index = sentence_index.search(prompt_embeddings, 6)

In [17]:
del sentence_index
del prompt_embeddings
_ = gc.collect()
libc.malloc_trim(0)

1

In [18]:
df = pd.read_parquet("/kaggle/input/wikipedia-20230701/wiki_2023_index.parquet",
                     columns=['id', 'file'])

In [19]:
wikipedia_file_data = []

for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    scr_idx = idx
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

## Save memory - delete df since it is no longer necessary
del df
_ = gc.collect()
libc.malloc_trim(0)

  0%|          | 0/200 [00:00<?, ?it/s]

1

In [20]:
wiki_text_data = []

for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

    _df_temp = _df[_df['id'].isin(_id)].copy()
    del _df
    _ = gc.collect()
    libc.malloc_trim(0)
    wiki_text_data.append(_df_temp)
wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
_ = gc.collect()

  0%|          | 0/27 [00:00<?, ?it/s]

In [21]:
processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values)


  0%|          | 0/1120 [00:00<?, ?it/s]

  0%|          | 0/1120 [00:00<?, ?it/s]

In [22]:
wiki_data_embeddings = model.encode(processed_wiki_text_data.text,
                                    batch_size=BATCH_SIZE,
                                    device=DEVICE,
                                    show_progress_bar=True,
                                    convert_to_tensor=True,
                                    normalize_embeddings=True)#.half()
wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()

Batches:   0%|          | 0/1991 [00:00<?, ?it/s]

In [23]:
_ = gc.collect()


In [24]:
question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
question_embeddings = question_embeddings.detach().cpu().numpy()

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [25]:
## Parameter to determine how many relevant sentences to include
NUM_SENTENCES_INCLUDE = 22

## List containing just Context
contexts = []

for r in tqdm(trn.itertuples(), total=len(trn)):

    prompt_id = r.Index

    prompt_indices = processed_wiki_text_data[processed_wiki_text_data['document_id'].isin(wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values)].index.values

    if prompt_indices.shape[0] > 0:
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        prompt_index.add(wiki_data_embeddings[prompt_indices])

        context = ""
        
        ## Get the top matches
        ss, ii = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
        for _s, _i in zip(ss[prompt_id], ii[prompt_id]):
            context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + " "
        
    contexts.append(context)

  0%|          | 0/200 [00:00<?, ?it/s]

In [26]:
trn['context'] = contexts

In [27]:
trn[["prompt", "context", "A", "B", "C", "D", "E"]].to_csv("./test_context.csv", index=False)

In [28]:
del model
del wiki_data_embeddings
del question_embeddings
del contexts

gc.collect()
torch.cuda.empty_cache()

In [29]:
gc.collect()
torch.cuda.empty_cache()

## Load data back

In [30]:
test_df = pd.read_csv("test_context.csv")
test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[:1500]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'

## Llama Inference

In [31]:
class CFG:
    base_model = "/kaggle/input/llama2-7b-hf/Llama2-7b-hf"
    seed = 42

In [32]:
def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state


random_state = set_seed(CFG.seed)

In [33]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(CFG.base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [34]:
class LlmseDataset(torch.utils.data.Dataset):
    def __init__(self, df, is_train=False, aug_prob=0.8):
        self.df = df
        self.is_train = is_train
        self.aug_prob = aug_prob
        self.option_to_index = {option: idx for idx, option in enumerate('ABCDE')}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        example = self.df.iloc[idx]

        first_sentence = [example['prompt']] * 5
        second_sentences = [example[option] for option in 'ABCDE']
        if len(example['prompt']) >= 2299:
            # Tokenize prompt with truncation
            tokenized_prompt = tokenizer(
                [example['prompt']],
                truncation=True,
                max_length=556,  
                return_tensors="pt",
                padding=True,
            )
        else:
            tokenized_prompt = tokenizer(
                [example['prompt']],
                #truncation=True,
                #max_length=556,  
                return_tensors="pt",
            )

        # Tokenizing all options together:
        second_sentences = [example[option] for option in 'ABCDE']
        tokenized_options = tokenizer(second_sentences, truncation=True, padding=True, return_tensors="pt")

        # Cat them with the prompt tokens one by one:
        tokenized_example = {}
        for key in tokenized_prompt.keys():
            # Here we assume the last dimension is the token dimension
            prompt_tokens = tokenized_prompt[key]
            tokenized_example[key] = torch.cat([torch.cat((prompt_tokens, tokenized_options[key][option_idx][None, :]), dim=-1) for option_idx in range(5)], dim=0)
        
        tokenized_example['label'] = self.option_to_index[example['answer']]
        return tokenized_example

In [35]:
val_ds = LlmseDataset(test_df, is_train=False)

def convert_to_feats(ds):
    return [ds[i] for i in range(len(ds))]

val_features = convert_to_feats(val_ds)

# train_dataset = Dataset.from_dict({k: [dic[k] for dic in train_features] for k in train_features[0]})
test_dataset = Dataset.from_dict({k: [dic[k] for dic in val_features] for k in val_features[0]})
len(val_ds[28]['input_ids'][0])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


657

In [36]:
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [37]:
llama_model = prepare_model_for_kbit_training(llama_model)
# llama_model

In [38]:
from transformers.modeling_outputs import SequenceClassifierOutput

def get_dropouts(num: int, start_prob: float, increment: float):
    return [nn.Dropout(start_prob + (increment * i)) for i in range(num)]

class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings


class CustomLlamaModel(nn.Module):
    def __init__(self, backbone, num_labels):
        super(CustomLlamaModel, self).__init__()
        # Using Huggingface automodel as base model
        self.model = backbone
        self.model.config.use_cache = False
        self.config = self.model.config
        self.num_labels = num_labels

        # self.pooler = ContextPooler(self.config)
        self.pooler = MeanPooling()
        self.dense = nn.Linear(self.config.hidden_size, 1024)

        # self.dropouts = get_dropouts(num=5, start_prob=config.hidden_dropout_prob - 0.02, increment=0.01)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(self.config.hidden_size, 1, bias=False)


    def forward(self, input_ids, attention_mask, token_type_ids=None, position_ids=None, head_mask=None,
                inputs_embeds=None, labels=None):
        num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]
        batch_size = input_ids.shape[0] if input_ids is not None else inputs_embeds.shape[0]

        flat_input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
        flat_position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
        flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
        flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
        flat_inputs_embeds = (
            inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
            if inputs_embeds is not None
            else None
        )

        outputs = self.model.model(input_ids=flat_input_ids, attention_mask=flat_attention_mask, output_attentions=False,
            output_hidden_states=True)

        last_hidden_state = outputs[1][-1]

        x = self.pooler(last_hidden_state, flat_attention_mask)
        x = self.dropout(x)
        logits = self.classifier(x)
        reshaped_logits = logits.view(batch_size, num_choices)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(reshaped_logits, labels)

        if self.model.config.output_attentions:
            attentions = outputs.attentions
        else:
            attentions = None

        if self.model.config.output_hidden_states:
            hidden_states = outputs.hidden_states
        else:
            hidden_states = None

        return SequenceClassifierOutput(
            loss=loss,
            logits=reshaped_logits,
            hidden_states=hidden_states,
            attentions=attentions,
        )

In [39]:
peft_config = LoraConfig(
    r=12,
    lora_alpha=4,  # parameter for scaling
    target_modules=['up_proj', 'o_proj', 'q_proj', 'k_proj', 'v_proj', 'gate_proj', 'down_proj'],
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
)

model = get_peft_model(llama_model, peft_config)

In [40]:
custom_llama = CustomLlamaModel(model, 5)
# Load the model

state_dict = torch.load("/kaggle/input/llama-fine-tuned-context-v2/pytorch_model.bin")
custom_llama.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [41]:
del state_dict
del llama_model

gc.collect()
torch.cuda.empty_cache()

In [42]:
# import torch.quantization as tq

# custom_llama = tq.quantize_dynamic(custom_llama, {torch.nn.Linear}, dtype=torch.qint8)

In [43]:
def apk(actual, predicted, k=5):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """

    # requires all elements are unique
    assert (len(np.unique(predicted)) == len(predicted))

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=5):

    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

from sklearn.metrics import top_k_accuracy_score
def metrics4trainer(pred):
    pred_logits = pred.predictions
    true_labels = pred.label_ids
    top1 = top_k_accuracy_score(true_labels, pred_logits, k=1)
    top3 = top_k_accuracy_score(true_labels, pred_logits, k=3)
    pred_args = np.argsort(-pred_logits)
    map3 = mapk(true_labels.reshape(-1, 1), pred_args.reshape(-1, 5), k=3)
    return {"top1": top1, "top3": top3, "map3":map3}

In [44]:
training_args = TrainingArguments(
    output_dir='op',
    overwrite_output_dir = True,
    per_device_eval_batch_size=1,
    remove_unused_columns=False,
    logging_steps=40,
    eval_steps=40,
    save_steps=40,
    evaluation_strategy='steps',
    fp16=True,
    bf16=False,
    report_to="none",
    metric_for_best_model='map3',
    seed=42,
#     fsdp=True
)

In [45]:
trainer = Trainer(
    model=custom_llama,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    eval_dataset=test_dataset,
    train_dataset=test_dataset,
#     is_model_parallel=True
    )

In [46]:
outputs = trainer.predict(test_dataset)
llama_softmax_outputs = softmax(outputs.predictions, axis=1)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [47]:
del trainer
del custom_llama
del model
del test_dataset
del val_ds
# del state_dict
# del llama_model

gc.collect()
torch.cuda.empty_cache()

In [48]:
gc.collect()
torch.cuda.empty_cache()

## Deberta Inference

In [49]:
test_df = pd.read_csv("test_context.csv")
test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
test_df["prompt"] = test_df["context"].apply(lambda x: x[:2100]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'

In [50]:
model_dir = "/kaggle/input/open-book-model/checkpoints_0.94583/checkpoints_0.94583/checkpoint-925"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_aff

In [51]:
# We'll create a dictionary to convert option names (A, B, C, D, E) into indices and back again
options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example

In [52]:
@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [53]:
tokenized_test_dataset = Dataset.from_pandas(test_df[['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer']].drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [54]:
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / e_x.sum(axis=-1, keepdims = True)    

In [55]:
%%time

test_predictions = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions.append(outputs.logits.cpu().detach())

test_predictions = torch.cat(test_predictions)
test_predictions = test_predictions.numpy()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


CPU times: user 1min 21s, sys: 62.8 ms, total: 1min 21s
Wall time: 1min 21s


In [56]:
%%time
model_dir = "/kaggle/input/how-to-train-open-book-model-part-1/model_v2/"
model_dir = "/kaggle/input/llm-science-run-context-2"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

CPU times: user 4.11 s, sys: 1.77 s, total: 5.88 s
Wall time: 17.5 s


DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_aff

In [57]:
test_predictions2 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions2.append(outputs.logits.cpu().detach())

test_predictions2 = torch.cat(test_predictions2)
test_predictions2 = test_predictions2.numpy()

test_predictions = softmax (test_predictions) + softmax(test_predictions2) + llama_softmax_outputs

predictions_as_ids = np.argsort(-test_predictions, 1)

predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
# predictions_as_answer_letters[:3]

predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

In [58]:
submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)

In [59]:
submission.head()

,id,prediction
0,0,D B E
1,1,A B E
2,2,A C D
3,3,C A B
4,4,D A C
